In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.stats import norm
from figaro.utils import rejection_sampler
from figaro.cosmology import CosmologicalParameters
from figaro.load import _find_redshift
from figaro import plot_settings

In [2]:
true_H0 = 40. # km/s/Mpc (Deliberately off value)
omega = CosmologicalParameters(true_H0/100., 0.315, 0.685, -1., 0.)
n_draws_set = 1
n_draws_event = 2000
norm_dist = norm()

In [15]:
# Mass distribution
def evolving_gaussian(m, dL):
    mu  = mean(dL)
    sigma = std(dL)
    return np.exp(-0.5 * ((m - mu) / sigma) ** 2) / (np.sqrt(2 * np.pi) * sigma)

def mean(dL, dMddL = 30./1000., offset = 5.):
    return dL*dMddL + offset

def std(dL, dMddL = 8./1000., offset = 1.):
    return dL*dMddL + offset

# dL distribution
def gaussian(dL, mu = 1600, sigma = 400):
    return np.exp(-0.5 * ((dL - mu) / sigma) ** 2) / (np.sqrt(2 * np.pi) * sigma)

# Generate samples from double gaussian distribution
def generate_event_parameters_double_gaussian(n_draws_set, n_draws_event):
    event_param_list = []
    for _ in tqdm(range(n_draws_set)):
        dL = rejection_sampler(n_draws_event, gaussian, [0, 5000])
        M  = np.array([mean(d) + std(d) * norm_dist.rvs() for d in dL])
        z  = np.array([_find_redshift(omega, d) for d in dL])
        Mz = M * (1 + z)
        event_param_list.append([Mz, dL])
    return np.array(event_param_list[0])

In [16]:
# Mass distribution
def PLpeak(m, alpha = -2., mmin = 5., mmax = 70., mu = 30., sigma = 4., w = 0.2):
    norm_pl = (1 - alpha) / (mmin ** (alpha + 1) - mmax ** (alpha + 1))
    pl      = norm_pl * m ** alpha
    peak    = np.exp(-0.5 * ((m - mu) / sigma) ** 2) / (np.sqrt(2 * np.pi) * sigma)
    return w * pl + (1 - w) * peak

# dL distribution
def dLsq(dL, dLmax = 5000):
    return 3 * dL ** 2 / dLmax ** 3

# Generate samples from realistic distribution
def generate_event_parameters_realistic(n_draws_set, n_draws_event):
    event_param_list = []
    for _ in tqdm(range(n_draws_set)):
        dL = rejection_sampler(n_draws_event, dLsq, [0, 5000])
        M  = rejection_sampler(n_draws_event, PLpeak, [5,70])
        z  = np.array([_find_redshift(omega, d) for d in dL])
        Mz = M * (1 + z)
        event_param_list.append([Mz, dL])
    return np.array(event_param_list[0])

In [39]:
def generate_event_posterior_samples(event_param, Mz_measure_uncer = 5, dL_measure_uncer = 50):
    Mz, dL = event_param
    Mz_obs_mean = Mz + Mz_measure_uncer * norm_dist.rvs()
    dL_obs_mean = dL + dL_measure_uncer * norm_dist.rvs()
    Mz_samples = np.random.normal(Mz_obs_mean, Mz_measure_uncer, 1000)
    dL_samples = np.random.normal(dL_obs_mean, dL_measure_uncer, 1000)
    return np.array([Mz_samples, dL_samples])